# LangChain and Google's Gemini

In [ ]:
# pip install -q langchain-google-genai

In [ ]:
# to upgrade
# !pip install --upgrade -q langchain-google-genai

In [ ]:
# pip show langchain-google-genai

In [ ]:
# pip install -q google-generativeai

In [ ]:
# Load the API key
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

In [ ]:
# If you can't load the API key, you can provide it manually
import getpass
import os
if 'MY_GOOGLE_API_KEY' not in os.environ:
    os.environ['MY_GOOGLE_API_KEY'] = getpass.getpass('Provide your Google API Key: ')

In [ ]:
import google.generativeai as genai
for model in genai.list_models():
    print(model.name)

## Integrating Gemini with LangChain

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model='gemini-pro', temperature=0.9)
response = llm.invoke('Write a paragraph about life on Mars in year 2100.')
print(response.content)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
llm = ChatGoogleGenerativeAI(model='gemini-pro')

prompt = PromptTemplate.from_template('You are a content creator. Write me a tweet about {topic}')
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True
)

topic = 'Why will AI change the World'
response = chain.invoke(input=topic)
print(response)

In [ ]:
print(response['topic'])
print()
print(response['text'])

# System Prompt and Streaming

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

llm = ChatGoogleGenerativeAI(model='gemini-pro', convert_system_message_to_human=True)

output = llm.invoke(
    [
        SystemMessage(content='Answer only YES or NO in French.'),
        HumanMessage(content='Is cat a mammal?')
    ]
)
output.content

## Streaming

In [ ]:
llm = ChatGoogleGenerativeAI(model='gemini-pro', temperature=0)
prompt = 'Write a scientific paper outlining the mathematical foundation of our universe.'
response = llm.invoke(prompt)
print(response.content)

In [ ]:
for chunk in llm.stream(prompt):
    print(chunk.content, end='')
    print('-' * 100)

## Multimodal AI with Gemini Pro Vision

In [ ]:
# pip install -q pillow

In [ ]:
from PIL import Image
img = Image.open('match.jpg')
img

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model='gemini-pro-vision')
prompt = 'What is in this image?'
message = HumanMessage(
    content=[
        {'type': 'text', 'text': prompt},
        {'type': 'image_url', 'image_url': img}
    ]
)
response = llm.invoke([message])
print(response.content)

In [ ]:
def ask_gemini(text, image, model='gemini-pro-vision'):
    """
    
    :param text: The text query or prompt for llm  
    :param image: The image
    :param model: The name of the Gemini LLM model
    :return: 
    """
    llm = ChatGoogleGenerativeAI(model=model)
    prompt = 'What is in this image?'
    message = HumanMessage(
        content=[
            {'type': 'text', 'text': text},
            {'type': 'image_url', 'image_url': image}
        ]
    )
    response = llm.invoke([message])
    return response.content

In [ ]:
response = ask_gemini('What is this sport? How can I identify the sport in this picture?', image=img)
print(response)


In [ ]:
response = ask_gemini('How many players can you identify in each team?', image=img)
print(response)

In [ ]:
import requests
from IPython.display import Image
image_url = 'https://picsum.photos/id/40/4106/2806'
content = requests.get(image_url).content
image_data = Image(content)
image_data

In [ ]:
response = ask_gemini('Describe this image as detailed as possible', image_url)
print(response)

## Gemini Safety Settings

In [ ]:
from langchain_google_genai import HarmCategory, HarmBlockThreshold

llm2 = ChatGoogleGenerativeAI(
    model = 'gemini-pro',
    safety_settings={
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH
    }
)